In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions
spark = SparkSession.builder.getOrCreate()

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')
#%cd /gdrive

Mounted at /gdrive
/gdrive


In [4]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
! kaggle datasets download grouplens/movielens-20m-dataset
! unzip /content/movielens-20m-dataset.zip

 95% 185M/195M [00:02<00:00, 62.1MB/s]
100% 195M/195M [00:03<00:00, 68.2MB/s]


In [9]:
ratings_df = spark.read.csv('/content/rating.csv',header = True)
ratings_df.show( n =5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



### Data Cleaning


1.   Remove rows with no Rating
2.   Filter out movies or users with few ratings



In [12]:
ratings_df = ratings_df.filter(functions.col('rating').isNotNull())

In [13]:
# Filter out movies or users with few ratings
min_ratings = 50
movies_df = ratings_df.groupBy('movieId').count().filter(functions.col('count') >= min_ratings)
movies_df = movies_df.select('movieId')
ratings_df = ratings_df.join(movies_df, on='movieId', how='inner')
users_df = ratings_df.groupBy('userId').count().filter(functions.col('count') >= min_ratings)
users_df = users_df.select('userId')
ratings_df = ratings_df.join(users_df, on='userId', how='inner')

In [22]:
ratings_df.printSchema()
ratings_df = ratings_df.withColumn("userId", ratings_df["userId"].cast('int'))
ratings_df = ratings_df.withColumn("movieId", ratings_df["movieId"].cast('int'))
ratings_df = ratings_df.withColumn("rating", ratings_df["rating"].cast('float'))
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)



In [26]:
# Split the data into training and test sets
(train_df, test_df) = ratings_df.randomSplit([0.8, 0.2])
train_df = train_df.limit(100,000)

ConnectionRefusedError: ignored

In [24]:
# Define the ALS model
als = ALS(rank=50, maxIter=20, regParam=0.1, userCol='userId', itemCol='movieId', ratingCol='rating',
          coldStartStrategy='drop')

In [25]:
# Fit the ALS model to the training data
model = als.fit(train_df)

Py4JJavaError: ignored